In [ ]:
import random
from itertools import product

# 定义变量
defective_rate = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
purchase_costs = [2, 8, 12, 2, 8, 12, 8, 12]
inspection_costs = [1, 1, 2, 1, 1, 2, 1, 2]

assembly_cost = 8
assembly_inspection_cost = 4
disassembly_semi_cost = 6
disassembly_final_cost = 10
market_loss = 40

import random

# 随机判断是否损坏
def random_defective(rate):
    return random.random() < rate

# 零件的检测逻辑
def inspect_part(part_index, inspect, defective_rate, purchase_costs, inspection_costs):
    cost = purchase_costs[part_index]
    part_defective_rate = defective_rate[part_index]

    if inspect:
        cost += inspection_costs[part_index]
        if random_defective(part_defective_rate):
            return cost, 1.0  # 不合格, 次品率100%
        else:
            return cost, 0.0  # 合格, 次品率0%
    return cost, part_defective_rate  # 未检测，带入原始损坏概率

# 拆解并重新装配半成品或成品
def reassemble(total_cost, disassembly_cost):
    total_cost += disassembly_cost  # 拆解成本
    total_cost += assembly_cost     # 重新装配成本
    return total_cost

# 半成品的检测和装配逻辑
def assemble_semi_product(parts_indices, inspect_parts, inspect_semi, defective_rate, purchase_costs, inspection_costs, assembly_cost, assembly_inspection_cost, disassembly_semi_cost):
    total_cost = 0
    cumulative_defective_rate = 1.0  # 初始化为1，表示所有零件都合格


    # 检测每个零件
    for part_index in parts_indices:
        cost, part_defective_rate = inspect_part(part_index, inspect_parts[part_index], defective_rate, purchase_costs, inspection_costs)
        total_cost += cost
        cumulative_defective_rate *= (1 - part_defective_rate)  # 累积合格率

    cumulative_defective_rate = 1 - cumulative_defective_rate  # 最终半成品的次品率是 1 - 累积合格率

    # 装配成本
    total_cost += assembly_cost

    # 检测半成品
    if inspect_semi:
        total_cost += assembly_inspection_cost
        if random_defective(cumulative_defective_rate):
            total_cost = reassemble(total_cost, disassembly_semi_cost)
            return total_cost, 1.0  # 次品率100%
        else:
            return total_cost, 0.0  # 合格
    return total_cost, cumulative_defective_rate  # 未检测，带入累积次品率

# 成品的检测和装配逻辑
def assemble_final_product(inspect_parts, inspect_semi_products, inspect_final, defective_rate, purchase_costs, inspection_costs, assembly_cost, assembly_inspection_cost, disassembly_semi_cost, disassembly_final_cost):
    total_cost = 0
    cumulative_defective_rate = 1.0

    # 半成品的零件集合
    semi_products = [[0, 1, 2], [3, 4, 5], [6, 7]]

    # 检测每个半成品
    for semi_index, semi_product in enumerate(semi_products):
        inspect_semi = inspect_semi_products[semi_index]  # 对应的半成品是否检测
        cost, semi_defective_rate = assemble_semi_product(semi_product, inspect_parts, inspect_semi, defective_rate, purchase_costs, inspection_costs, assembly_cost, assembly_inspection_cost, disassembly_semi_cost)
        total_cost += cost
        cumulative_defective_rate *= (1 - semi_defective_rate)  # 累积合格率

    cumulative_defective_rate = 1 - cumulative_defective_rate  # 最终成品的次品率

    # 装配成品成本
    total_cost += assembly_cost

    # 检测成品
    if inspect_final:
        total_cost += assembly_inspection_cost
        if random_defective(cumulative_defective_rate):
            total_cost = reassemble(total_cost, disassembly_final_cost)
            return total_cost, 1.0  # 次品率100%
        else:
            return total_cost, 0.0  # 合格
    return total_cost, cumulative_defective_rate  # 未检测，带入累积次品率


# Corrected 'handle_returned_product' function with missing parameters added
def handle_returned_product(inspect_parts, inspect_semi_products, inspect_final):
    # Add missing parameters for assembly and disassembly costs, etc.
    total_cost, product_pass = assemble_final_product(
        inspect_parts=inspect_parts, 
        inspect_semi_products=inspect_semi_products, 
        inspect_final=inspect_final,
        defective_rate=defective_rate,
        purchase_costs=purchase_costs,
        inspection_costs=inspection_costs,
        assembly_cost=assembly_cost,
        assembly_inspection_cost=assembly_inspection_cost,
        disassembly_semi_cost=disassembly_semi_cost,
        disassembly_final_cost=disassembly_final_cost
    )

    # If the product is defective and undetected, add market loss
    if not product_pass:
        total_cost += market_loss
    return total_cost

# Corrected 'simulate_strategy' function with missing parameters added
def simulate_strategy(inspect_parts, inspect_semi_products, inspect_final, num_simulations=1000):
    total_cost = 0
    for _ in range(num_simulations):
        # Simulate the final product assembly and inspection process
        cost, product_pass = assemble_final_product(
            inspect_parts=inspect_parts, 
            inspect_semi_products=inspect_semi_products, 
            inspect_final=inspect_final,
            defective_rate=defective_rate,
            purchase_costs=purchase_costs,
            inspection_costs=inspection_costs,
            assembly_cost=assembly_cost,
            assembly_inspection_cost=assembly_inspection_cost,
            disassembly_semi_cost=disassembly_semi_cost,
            disassembly_final_cost=disassembly_final_cost
        )
        total_cost += cost

        # If the defective product enters the market
        if not product_pass:
            total_cost += market_loss

    # Calculate average cost
    return total_cost / num_simulations

# Function to find the optimal inspection strategy
def find_optimal_strategy():
    best_cost = float('inf')
    best_strategy = None

    # Iterate over all combinations of part, semi-product, and final inspections
    for inspect_parts in product([True, False], repeat=8):
        for inspect_semi_products in product([True, False], repeat=3):
            for inspect_final in [True, False]:
                # Simulate the strategy
                cost = simulate_strategy(
                    inspect_parts=inspect_parts, 
                    inspect_semi_products=inspect_semi_products, 
                    inspect_final=inspect_final
                )

                # Print results for each strategy
                print(f"零配件检测策略: {inspect_parts}, 半成品检测策略: {inspect_semi_products}, 成品检测: {inspect_final}, 平均成本: {cost}")

                # Compare cost to find the optimal strategy
                if cost < best_cost:
                    best_cost = cost
                    best_strategy = (inspect_parts, inspect_semi_products, inspect_final)

    return best_strategy, best_cost

# Find the optimal inspection strategy
best_strategy, best_cost = find_optimal_strategy()

print("最优检测策略:", best_strategy)
print("最低平均成本:", best_cost)

